In [1]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import subprocess
import pandas as pd
import openai
import nest_asyncio
import asyncio
import importlib.util

nest_asyncio.apply()

# Set OpenAI API key
with open('/content/drive/MyDrive/key.txt', 'r') as file:
    api_key = file.read().strip()

client = openai.AsyncOpenAI(api_key=api_key)

# Paths
folder_path = '/content/drive/MyDrive/Programs/'
csv_input_path = os.path.join(folder_path, 'program_rmse_results.csv')
results_df = pd.read_csv(csv_input_path)
program_number = len(results_df) + 1
new_code_filename = f"program{program_number}.py"

# Read best performing programs
best_programs = ['program7.py', 'program11.py', 'program13.py']
best_code_snippets = []

for prog in best_programs:
    prog_path = os.path.join(folder_path, prog)
    if os.path.exists(prog_path):
        with open(prog_path, 'r', encoding='utf-8') as file:
            best_code_snippets.append(file.read())

# Async function to get an improved version of the code
async def get_improved_code(best_codes):
    instruction = (
        "Improve the given Python code to achieve a lower RMSE score. "
        "Use techniques such as hyperparameter tuning, data preprocessing improvements, "
        "and model enhancements. Return ONLY the improved Python code without any explanations."
    )
    prompt_content = instruction + "\n\n" + "\n\n".join(best_codes)

    response = await client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt_content}],
        temperature=0.2
    )
    return response.choices[0].message.content.strip('` \n')

# Function to execute and test the generated code
def execute_and_test(filepath):
    try:
        spec = importlib.util.spec_from_file_location("generated_module", filepath)
        module = importlib.util.module_from_spec(spec)
        spec.loader.exec_module(module)
        rmse = module.CalculateRMSE()
        print(f"Execution successful. RMSE: {rmse:.4e}")
        return True, rmse
    except Exception as e:
        print(f"Error executing {filepath}: {e}")
        return False, str(e)

# Async function to correct buggy Python code
async def correct_code(buggy_code, error_message):
    correction_prompt = (
        "The following Python code produced an error:\n"
        f"Error: {error_message}\n"
        f"Code:\n{buggy_code}\n"
        "Correct the Python code to fix the error and ensure it runs successfully. "
        "Return ONLY the corrected Python code without any explanations."
    )
    response = await client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": correction_prompt}],
        temperature=0.2
    )
    return response.choices[0].message.content.strip('` \n')

# Main loop to generate, test, and correct improved code
async def main_loop(max_trials=5):
    improved_code = await get_improved_code(best_code_snippets)

    # Save the improved code
    code_path = os.path.join(folder_path, new_code_filename)
    with open(code_path, 'w', encoding='utf-8') as file:
        file.write(improved_code)
    print(f"Saved improved code to {new_code_filename}")

    trial = 1
    success = False

    while trial <= max_trials and not success:
        print(f"Trial {trial}: Running {new_code_filename}...")
        success, result = execute_and_test(code_path)

        if success:
            print(f"Success on trial {trial}. RMSE: {result:.4e}")
            break
        else:
            print(f"Trial {trial} error: {result}")
            improved_code = await correct_code(improved_code, result)
            with open(code_path, 'w', encoding='utf-8') as file:
                file.write(improved_code)

        trial += 1

    if not success:
        print(f"Failed after {max_trials} trials.")

# Run the loop
asyncio.run(main_loop())

Mounted at /content/drive
Saved improved code to program9.py
Trial 1: Running program9.py...
RMSE: 1.03e-05
R² Score: 0.48
Execution successful. RMSE: 1.0262e-05
Success on trial 1. RMSE: 1.0262e-05
